In [1]:
# 1. Importar bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch


C:\Users\Guilherme\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. Carregar dados
dataset_path = "trn.json"  # Caminho do dataset local
dataset = load_dataset("json", data_files=dataset_path, split="train")
dataset = dataset.train_test_split(test_size=0.1, seed=42)  # 90% treino, 10% teste

In [3]:
# 3. Usar modelo pequeno e rápido
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Definir token de padding

In [4]:

# 4. Processamento dos dados
def process_examples(examples):
    texts = [f"Descreva este produto: {title}\nDescrição: {content}"
             for title, content in zip(examples['title'], examples['content'])]

    tokenized = tokenizer(
        texts,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized


In [5]:
# 5. Processar datasets
train_dataset = dataset["train"].map(
    process_examples,
    batched=True,
    remove_columns=['title', 'content']
)

eval_dataset = dataset["test"].map(
    process_examples,
    batched=True,
    remove_columns=['title', 'content']
)


Map: 100%|████████████████████████████████████████████████████████████| 224862/224862 [00:24<00:00, 9347.74 examples/s]


In [22]:
# 6. Carregar e configurar modelo
model = AutoModelForCausalLM.from_pretrained(model_name)

# Definir o dispositivo correto (CPU ou ROCm para AMD)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
# 7. Configurar treinamento
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=5e-5,
    fp16=False,  # fp16 só funciona em GPUs NVIDIA
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="no",
    max_steps=1000
)


In [27]:
# 8. Criar trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


In [28]:
# 9. Executar treinamento
print("🚀 Iniciando treinamento...")
trainer.train()
print("✅ Treinamento concluído!")

🚀 Iniciando treinamento...


Epoch,Training Loss,Validation Loss
0,2.312500,2.104169


✅ Treinamento concluído!


In [50]:
# 10. Função de geração
def gerar_descricao(titulo):
    prompt = f"Descreva este produto: {titulo}\nDescrição:"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,
        temperature=0.5,
        do_sample=True, 
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Descrição:")[-1].strip()


In [62]:
# 11. Testar
print("\nExemplo de saída:")
descricao = gerar_descricao("Smartphone Android S23")
print(descricao) 


Exemplo de saída:
Smartphone Android S23 features a built-in Bluetooth connection for the Android 5.0 Marshmallow OS. The device is equipped with a 2.5 inch display and Bluetooth 4.0 support. The device is powered by a 4.5-inch 4.5 inch display with a 1.25 inch (1.25" x 1.25") battery. The phone is equipped with a 2.5 inch (1.25" x 1.25") battery and a 1
